In [7]:
# Add source directories to search path
import sys
src_paths = ["macros\\"]
for src_path in src_paths:
    if not src_path in sys.path:
        sys.path.append(src_path)



In [8]:
from jinja2 import Template

import sympy as sp
from sympy.printing.fortran import fcode

# Jinja2 template for a fypp macro for a binary functions
macro = '''
#:def {{interface_name}}_{{sn}}(dual_type, real_kind, is_pure=True, test_coverage=False)
    #!set elemental_purity = dnad.elemental_purity(is_pure, test_coverage)
    {% for fun in funs %}
        {{fun}}
    {% endfor %}
#:enddef
'''

# Jinja2 template for binary function for dual numbers inside a fypp macro
binary_fun_d = '''
    ${elemental_purity}$ function {{interface_name}}_{{usn}}_{{vsn}}(u, v) result(res)
        {{u_type}}, intent(in) :: u
        {{v_type}}, intent(in) :: v
        {{res_type}} :: res
        {{x_body}}
        {{dx_body}}
      #:if test_coverage == True
        {{interface_name}}_{{usn}}_{{vsn}}_counter = {{interface_name}}_{{usn}}_{{vsn}}_counter + 1
      #:endif
    end function'''

# Jinja2 template for a binary function for hyper-dual numbers inside a fypp macro
binary_fun_hd = '''
    ${elemental_purity}$ function {{interface_name}}_{{usn}}_{{vsn}}(u, v) result(res)
        {{u_type}}, intent(in) :: u
        {{v_type}}, intent(in) :: v
        {{res_type}} :: res
        integer :: j
        {{x_body}}
        {{dx_body}}
        {{ddx_body}}
      #:if test_coverage == True
        {{interface_name}}_{{usn}}_{{vsn}}_counter = {{interface_name}}_{{usn}}_{{vsn}}_counter + 1
      #:endif
    end function'''

binary_fun_d_tm = Template(binary_fun_d)
binary_fun_hd_tm = Template(binary_fun_hd)





In [9]:
funs = []
fun = binary_fun_d_tm.render(
    interface_name = 'add',
    usn = 'd',
    vsn = 'd',
    u_type = 'real(dp)',
    v_type = 'real(dp)',
    res_type = 'real(dp)',
    x_body = 'res = u + v',
    dx_body = 'res = u + v')
funs.append(fun)

fun = binary_fun_hd_tm.render(
    interface_name = 'add',
    usn = 'hd',
    vsn = 'hd',
    u_type = 'real(dp)',
    v_type = 'real(dp)',
    res_type = 'real(dp)',
    x_body = 'res = u + v',
    dx_body = 'res = u + v',
    ddx_body = 'res = u + v')
funs.append(fun)

macro_impl = Template(macro).render(
    interface_name = 'add',
    sn = 'd_d',
    funs = funs)

print(macro_impl)



#:def add_d_d(dual_type, real_kind, is_pure=True, test_coverage=False)
    #!set elemental_purity = dnad.elemental_purity(is_pure, test_coverage)
    
        
    ${elemental_purity}$ function add_d_d(u, v) result(res)
        real(dp), intent(in) :: u
        real(dp), intent(in) :: v
        real(dp) :: res
        res = u + v
        res = u + v
      #:if test_coverage == True
        add_d_d_counter = add_d_d_counter + 1
      #:endif
    end function
    
        
    ${elemental_purity}$ function add_hd_hd(u, v) result(res)
        real(dp), intent(in) :: u
        real(dp), intent(in) :: v
        real(dp) :: res
        integer :: j
        res = u + v
        res = u + v
        res = u + v
      #:if test_coverage == True
        add_hd_hd_counter = add_hd_hd_counter + 1
      #:endif
    end function
    
#:enddef
